<a href="https://colab.research.google.com/github/skj092/Computer_Vision_Lab/blob/main/Global_Wheat_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
main_dir = "/content/drive/MyDrive/ML"

# Downloading the dataset

In [5]:
!mv kaggle.json /root/.kaggle/

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [17]:
cd /content/drive/MyDrive/ML

/content/drive/MyDrive/ML


In [18]:
!mkdir /content/drive/MyDrive/ML/data

In [20]:
cd /content/drive/MyDrive/ML/data

/content/drive/MyDrive/ML/data


In [21]:
!kaggle competitions download -c global-wheat-detection

 98% 593M/607M [00:04<00:00, 92.0MB/s]
100% 607M/607M [00:04<00:00, 132MB/s] 


In [ ]:
!unzip global-wheat-detection.zip

In [23]:
!rm global-wheat-detection.zip

# Setting up Envieonment 

In [30]:
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
!git checkout v0.3.0

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

Cloning into 'vision'...
remote: Enumerating objects: 177820, done.
remote: Counting objects: 100% (10205/10205), done.
remote: Compressing objects: 100% (634/634), done.
remote: Total 177820 (delta 9612), reused 10083 (delta 9550), pack-reused 167615
Receiving objects: 100% (177820/177820), 350.71 MiB | 31.25 MiB/s, done.
Resolving deltas: 100% (159200/159200), done.
fatal: not a git repository (or any of the parent directories): .git


In [31]:
!pip uninstall opencv-python-headless -y

In [32]:
!pip install opencv-python-headless==4.1.2.30

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 1.5 MB/s 


In [7]:
root_dir = "/content/drive/MyDrive/ML"

In [18]:
# Importing necessary libraries 

import os 
from pathlib import Path 
import xml.etree.ElementTree as ET 
from torch.utils.data import Dataset
from PIL import Image
import torch 
import albumentations as A
# from albumentations.pytorch.transforms import ToTensorV2
import numpy as np
import cv2 
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision
import pandas as pd 
import ast
from sklearn import model_selection

# these are the helper libraries imported.
from engine import train_one_epoch, evaluate
import utils
# import transforms as T

# Define Transform

In [9]:
import torchvision.transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

# Dataset and DataLoader

In [10]:
class WheatDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform 
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        image_id, bboxes = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, image_id+'.jpg')
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        boxes = []
        for bbox in bboxes:
            x1 = bbox[0]
            y1 = bbox[1]
            x2 = x1+bbox[2]
            y2 = y1+bbox[3]
            bbox = [x1, y1, x2, y2]
            boxes.append(bbox)
        labels = torch.ones((len(boxes),), dtype=torch.int64)
        bboxes = torch.as_tensor(boxes, dtype=torch.float32)
        target = {'boxes': bboxes, 'labels': labels}
        return image, target

In [11]:
df = pd.read_csv(os.path.join(root_dir, 'data/train.csv'))
df['bbox'] = df['bbox'].apply(ast.literal_eval)
df = df.groupby('image_id')['bbox'].apply(list).reset_index(name='bboxes')

# Model

In [12]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_object_detection_model(num_classes):
    # load a model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

# Testing On One Batch

In [16]:
train_dir = os.path.join(root_dir, 'data/train')
ds = WheatDataset(df, train_dir, transform=get_transform(train=True))
dl = DataLoader(ds, batch_size=2, shuffle=True, collate_fn=utils.collate_fn)
images,targets = next(iter(dl))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
model = get_object_detection_model(2)
predictions = model(images, targets)
predictions

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


{'loss_box_reg': tensor(0.3618, grad_fn=<DivBackward0>),
 'loss_classifier': tensor(0.8381, grad_fn=<NllLossBackward0>),
 'loss_objectness': tensor(4.3046, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'loss_rpn_box_reg': tensor(0.1343, grad_fn=<DivBackward0>)}

In [21]:
def prepare_data(df):
    df['bbox'] = df['bbox'].apply(ast.literal_eval)
    df = df.groupby('image_id')['bbox'].apply(list).reset_index(name='bboxes')
    return df

In [31]:
# config 

device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
df = pd.read_csv(os.path.join(root_dir, 'data/train.csv'))
df_train, df_valid = model_selection.train_test_split(df, test_size=0.1, random_state=42)
df_train = prepare_data(df_train)
df_valid = prepare_data(df_valid)

# datasets 
image_dir = os.path.join(root_dir, 'data/train')
train_ds = WheatDataset(df_train, image_dir, transform=get_transform(train=True))
valid_ds = WheatDataset(df_valid, image_dir, transform=get_transform(train=False))

# dataloaders
train_dl = DataLoader(train_ds, batch_size=2, shuffle=True, collate_fn=utils.collate_fn)
valid_dl = DataLoader(valid_ds, batch_size=2, shuffle=False, collate_fn=utils.collate_fn)

# model
model = get_object_detection_model(2)
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [34]:
# train the model
num_epochs = 10
for epoch in range(num_epochs):
    # training for one epoch
    train_one_epoch(model, optimizer, train_dl, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the validation set
    evaluate(model, valid_dl, device=device)

/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


KeyboardInterrupt: ignored